In [11]:
import adlfs
import glob

In [12]:
# use read only token
sas_token = 'se=2023-04-15T00%3A35Z&sp=racwdl&sv=2021-06-08&sr=c&skoid=598b2582-4d1d-4c4e-9bb2-3ad571b791b5&sktid=f6b6dd5b-f02f-441a-99a0-162ac5060bd2&skt=2023-04-08T00%3A35%3A28Z&ske=2023-04-15T00%3A35%3A00Z&sks=b&skv=2021-06-08&sig=wlicNZ52wri%2BNf8jSon/QYnlxYXtuimAHmeBlEf5M5o%3D'

tile_names = [tile[-5:] for tile in adlfs.AzureBlobFileSystem(account_name="snowmelt", credential=sas_token, anon=False).ls('snowmelt/eric/MGRS/')]

In [13]:
years = ['allyears_median','allyears_std','allyears_corr_strength','2015_median','2016_median','2017_median','2018_median','2019_median','2020_median','2021_median','2022_median']
UTM_zone = [10,11,12,13]
resolution = 80

In [4]:
for year in years:
    for zone in UTM_zone:
        f = open(f'vrts/filenames/files_{year}_utm{zone}.txt','w')
        tile_names_zone = [s for s in tile_names if f'{zone}' in s]
        for tile in tile_names_zone:
            tif = f"https://snowmelt.blob.core.windows.net/snowmelt/eric/MGRS/{tile}/runoff_onset_{tile}_{year}_{resolution}m.tif"
            f.write(f'{tif}\n')
        f.close()

In [5]:
for year in years:
    if (year == 'allyears_std') | (year == 'allyears_corr_strength'):
        nodata = "nan"
    else:
        nodata = '-32768'
    for zone in UTM_zone:
        !gdalbuildvrt -input_file_list $f'vrts/filenames/files_{year}_utm{zone}.txt' $f'vrts/vrt_{year}_utm{zone}.vrt' -vrtnodata $nodata
        # ok so i should do a gdalwarp of the vrt to equal area projection but write to temp. then combine all
        #!gdalwarp -t_srs $"+proj=laea +lon_0=-113.8623047 +lat_0=39.1768337 +datum=WGS84 +units=m +no_defs" $f'vrts/vrt_{year}_utm{zone}.vrt' $f'vrts/vrt_{year}_utm{zone}_proj.vrt'

0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...ERROR 1: HTTP error code : 404
Warning 1: Can't open https://snowmelt.blob.core.windows.net/snowm

In [15]:
for year in years:
    if (year == 'allyears_std') | (year == 'allyears_corr_strength'):
        nodata = "nan"
    else:
        nodata = '-32768'
    for zone in UTM_zone:
        # ok so i should do a gdalwarp of the vrt to equal area projection but write to temp. then combine all
        !gdalwarp -t_srs $"+proj=latlon" $f'vrts/vrt_{year}_utm{zone}.vrt' $f'vrts/vrt_{year}_utm{zone}_proj.vrt'
    #!gdal_merge.py -o $f'/tmp/tif_{year}_merged.tif' $f'/tmp/tif_{year}_utm'* 

Creating output file that is 9333P x 16863L.
Processing vrts/vrt_allyears_median_utm10.vrt [1/1] : 0Using internal nodata values (e.g. -32768) for image vrts/vrt_allyears_median_utm10.vrt.
Copying nodata values from source vrts/vrt_allyears_median_utm10.vrt to destination vrts/vrt_allyears_median_utm10_proj.vrt.
...10...20...30...40...50...60...70...80...90...100 - done.
Creating output file that is 11245P x 21790L.
Processing vrts/vrt_allyears_median_utm11.vrt [1/1] : 0Using internal nodata values (e.g. -32768) for image vrts/vrt_allyears_median_utm11.vrt.
Copying nodata values from source vrts/vrt_allyears_median_utm11.vrt to destination vrts/vrt_allyears_median_utm11_proj.vrt.
...10...20...30...40...50...60...70...80...90...100 - done.
Creating output file that is 11295P x 23092L.
Processing vrts/vrt_allyears_median_utm12.vrt [1/1] : 0Using internal nodata values (e.g. -32768) for image vrts/vrt_allyears_median_utm12.vrt.
Copying nodata values from source vrts/vrt_allyears_median_ut

In [1]:
# import rioxarray as rxr
# vrt = rxr.open_rasterio('tifs/vrt_allyears_median_merged.vrt')
# vrt_coarsen = vrt.coarsen(x=10,y=10,boundary='trim').mean()
# vrt_coarsen
# vrt_coarsen.plot()

In [ ]:
# the following code combines vrts from across different UTM zones for each layer

In [16]:
for year in years:
    vrt_filenames = glob.glob(f'vrts/*{year}*proj.vrt') #*proj
    f = open(f'vrts/vrt_filelist_{year}.txt','w')
    for vrt_filename in vrt_filenames:
        f.write(f'{vrt_filename}\n')
    f.close()

In [17]:
for year in years:
    if (year == 'allyears_std') | (year == 'allyears_corr_strength'):
        nodata = "nan"
    else:
        nodata = '-32768'
        
    merged_vrt_filename = f'vrts/merged_{year}.vrt'
    text_file = f'vrts/vrt_filelist_{year}.txt' 
    !gdalbuildvrt -input_file_list $text_file $merged_vrt_filename -vrtnodata $nodata

0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.


In [18]:
import dask_gateway
cluster = dask_gateway.GatewayCluster()
client = cluster.get_client()
cluster.adapt(minimum=90, maximum=100)
print(client.dashboard_link)

https://pccompute.westeurope.cloudapp.azure.com/compute/services/dask-gateway/clusters/prod.66f7af33ab5e4e15b387add98a739dc0/status


In [19]:
!conda create -y -n azure python
!conda activate azure
!pip install azure-cli

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.12.0
  latest version: 23.3.1

Please update conda by running

    $ conda update -n base conda



## Package Plan ##

  environment location: /srv/conda/envs/azure

  added / updated specs:
    - python


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    _libgcc_mutex-0.1          |      conda_forge           3 KB  conda-forge
    _openmp_mutex-4.5          |            2_gnu          23 KB  conda-forge
    bzip2-1.0.8                |       h7f98852_4         484 KB  conda-forge
    ca-certificates-2022.12.7  |       ha878542_0         143 KB  conda-forge
    ld_impl_linux-64-2.40      |       h41732ed_0         688 KB  conda-forge
    libexpat-2.5.0             |       hcb278e6_1          76 KB  conda-forge
    libffi-3.4.2               |       h7f98852_5          57 KB  conda-forge
    l

In [20]:
!az login

To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code C2GKDLYMC to authenticate.
[
  {
    "cloudName": "AzureCloud",
    "homeTenantId": "f6b6dd5b-f02f-441a-99a0-162ac5060bd2",
    "id": "c8a2ce97-0a74-4a49-ba34-7c8d529837a6",
    "isDefault": true,
    "managedByTenants": [],
    "name": "rob5 main",
    "state": "Enabled",
    "tenantId": "f6b6dd5b-f02f-441a-99a0-162ac5060bd2",
    "user": {
      "name": "egagli@uw.edu",
      "type": "user"
    }
  }
]


In [23]:
#for year in years:
#    merged_tif_filename = f'vrts/merged_{year}.tif'
#    !gdalwarp -t_srs EPSG:32611 vrts/vrt_$year_utm??.vrt $merged_tif_filename -of COG

#for year in years:
#    merged_vrt_filename = f'tifs/merged_{year}.tif'
#    text_file = f'vrts/vrt_filelist_{year}.txt'
#    !gdalwarp -t_srs EPSG:32611 --optfile $text_file $merged_vrt_filename -of COG
    
for year in years:
    merged_vrt_filename = f'vrts/merged_{year}.vrt'
    merged_tif_filename = f'/tmp/merged_{year}.tif'
    blob_tif_filename = f'eric/products/merged_{year}.tif'
    !gdal_translate $merged_vrt_filename $merged_tif_filename -of COG
    
    !az storage blob upload \
    --account-name snowmelt \
    --container-name snowmelt \
    --name $blob_tif_filename \
    --file $merged_tif_filename \
    --output table
    
    !rm $merged_tif_filename

#for year in years:
#    !gdalwarp -t_srs $"+proj=laea +lon_0=-113.8623047 +lat_0=39.1768337 +datum=WGS84 +units=m +no_defs" vrts/vrt_$year*.vrt tifs/vrt_merged_$year.tif -of COG

Input file size is 33280, 22909
0...10...20...30...40...50...60...70...80...90...100 - done.

There are no credentials provided in your command and environment, we will query for account key for your storage account.
It is recommended to provide --connection-string, --account-key or --sas-token in your command as credentials.

You also can add `--auth-mode login` in your command to use Azure Active Directory (Azure AD) for authorization if your login account is assigned required RBAC roles.
For more information about RBAC roles in storage, visit https://docs.microsoft.com/azure/storage/common/storage-auth-aad-rbac-cli.

In addition, setting the corresponding environment variables can avoid inputting credentials in your command. Please use --help to get more information about environment variable usage.
Finished[#############################################################]  100.0000%
Table output unavailable. Use the --query option to specify an appropriate query. Use --debug for more 

2023-04-10 22:04:29,940 - distributed.client - ERROR - Failed to reconnect to scheduler after 30.00 seconds, closing client


.

Exception in callback None()
handle: <Handle cancelled>
Traceback (most recent call last):
  File "/srv/conda/envs/notebook/lib/python3.10/site-packages/tornado/iostream.py", line 1391, in _do_ssl_handshake
    self.socket.do_handshake()
  File "/srv/conda/envs/notebook/lib/python3.10/ssl.py", line 1342, in do_handshake
    self._sslobj.do_handshake()
ssl.SSLEOFError: EOF occurred in violation of protocol (_ssl.c:997)

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/srv/conda/envs/notebook/lib/python3.10/asyncio/events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "/srv/conda/envs/notebook/lib/python3.10/site-packages/tornado/platform/asyncio.py", line 189, in _handle_events
    handler_func(fileobj, events)
  File "/srv/conda/envs/notebook/lib/python3.10/site-packages/tornado/iostream.py", line 696, in _handle_events
    self._handle_read()
  File "/srv/conda/envs/notebook/lib/python3.10

10...20...30...40...50...60...70...80...90...100 - done.

There are no credentials provided in your command and environment, we will query for account key for your storage account.
It is recommended to provide --connection-string, --account-key or --sas-token in your command as credentials.

You also can add `--auth-mode login` in your command to use Azure Active Directory (Azure AD) for authorization if your login account is assigned required RBAC roles.
For more information about RBAC roles in storage, visit https://docs.microsoft.com/azure/storage/common/storage-auth-aad-rbac-cli.

In addition, setting the corresponding environment variables can avoid inputting credentials in your command. Please use --help to get more information about environment variable usage.
Finished[#############################################################]  100.0000%
Table output unavailable. Use the --query option to specify an appropriate query. Use --debug for more info.
Input file size is 33280, 2290